In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import clodius.tiles.vcf as ctvc

In [3]:
import os.path as op
import pandas as pd

chromsizes = []
with open('/Users/pete/projects/negspy/negspy/data/b37/chromInfo.txt') as f:
    for line in f:
        chromsizes += [line.strip().split('\t')]

In [4]:
import numpy as np
def chromsizes_array_to_series(chromsizes):
    """
    Convert an array of [[chrname, size]...] values to a series
    indexed by chrname with size values
    """
    chrnames = [c[0] for c in chromsizes]
    chrvalues = [c[1] for c in chromsizes]

    return pd.Series(np.array([int(c) for c in chrvalues]), index=chrnames)

cs_df = chromsizes_array_to_series(chromsizes)

In [5]:
%%time
filename = '/Users/pete/projects/resgen-server/media/s3/resgen-test/test.resgen.io/Io_XeSdtR2SOC01FapZWIQ/HG002_SVs_Tier1_v0.6.vcf.gz..'
index_filename = '/Users/pete/projects/resgen-server/media/s3/resgen-test/test.resgen.io/Io_XeSdtR2SOC01FapZWIQ/HG002_SVs_Tier1_v0.6.vcf.gz.tbi..'

tsinfo = ctvc.tileset_info(
    filename,
    chromsizes = cs_df
)

CPU times: user 203 µs, sys: 1 µs, total: 204 µs
Wall time: 212 µs


In [6]:
%%time

ctvc.single_tile(
    filename,
    index_filename,
    cs_df,
    tsinfo,
    z=15, x=0,
    max_tile_width=1e5
)

CPU times: user 32 ms, sys: 7.64 ms, total: 39.6 ms
Wall time: 77.3 ms


[{'uid': 'HG2_Ill_svaba_2',
  'importance': 0.5016339882894041,
  'xStart': 10402.0,
  'xEnd': 10440.0,
  'chrOffset': 0.0,
  'fields': ['1', 10402, 10440]},
 {'uid': 'HG3_Ill_GATKHC_3',
  'importance': 0.8019920899898374,
  'xStart': 10414.0,
  'xEnd': 10440.0,
  'chrOffset': 0.0,
  'fields': ['1', 10414, 10440]},
 {'uid': 'HG4_Ill_GATKHC_3',
  'importance': 0.09966513382502162,
  'xStart': 10615.0,
  'xEnd': 10637.0,
  'chrOffset': 0.0,
  'fields': ['1', 10615, 10637]},
 {'uid': 'HG2_Ill_250bpfermikitraw_2',
  'importance': 0.590269873965928,
  'xStart': 54711.0,
  'xEnd': 54712.0,
  'chrOffset': 0.0,
  'fields': ['1', 54711, 54712]}]

In [55]:
%%time

ctvc.tiles(
    filename,
    ['x.15.0'],
    index_filename,
    cs_df,
    max_tile_width=1e5
)

chrom, start, end 1 0.0 94473.08291625977
CPU times: user 33.6 ms, sys: 3.78 ms, total: 37.3 ms
Wall time: 38.7 ms


[('x.15.0',
  [{'uid': 'HG2_Ill_svaba_2',
    'importance': 0.6561968176043511,
    'xStart': 10402.0,
    'xEnd': 10440.0,
    'chrOffset': 0.0,
    'fields': ['1', 10402, 10440]},
   {'uid': 'HG3_Ill_GATKHC_3',
    'importance': 0.10543784967206138,
    'xStart': 10414.0,
    'xEnd': 10440.0,
    'chrOffset': 0.0,
    'fields': ['1', 10414, 10440]},
   {'uid': 'HG4_Ill_GATKHC_3',
    'importance': 0.22633892362040364,
    'xStart': 10615.0,
    'xEnd': 10637.0,
    'chrOffset': 0.0,
    'fields': ['1', 10615, 10637]},
   {'uid': 'HG2_Ill_250bpfermikitraw_2',
    'importance': 0.13035525917051494,
    'xStart': 54711.0,
    'xEnd': 54712.0,
    'chrOffset': 0.0,
    'fields': ['1', 54711, 54712]}])]

In [7]:
from pysam import VariantFile

vcf = VariantFile(filename, index_filename=index_filename)  # auto-detect input format

rec = list(vcf.fetch('1',0,1e5))[1]